In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import os

In [2]:
max_dataset_size = 1000000;
waveform_length = 72;
nb_of_elements = 150000;
nb_of_elements = 150;

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
nb_of_datasets = 45;
snr_from = 20;
snr_to = 100;
max_shift = 18;
use_horizontal_flip = False;
use_vertical_flip = False;
filter_type = 'high';
sampling_rate = 24000;
passband = np.array([100], dtype=int);
order = 1;



In [3]:
root_folder = os.path.dirname(os.getcwd())
path_to_recordings = path.join(root_folder,'data/synthesized');
path_to_ground_truth_data = path.join(root_folder,'data/synthesized');
path_to_train_data = path.join(root_folder,'data/detect_train.npy');
path_to_train_labels = path.join(root_folder,'data/detect_labels.npy');
path_to_noise_data = path.join(root_folder,'data/noise_data.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn
transform_list =[cnn.FilterSignalUsingButtersWorth(filter_type, sampling_rate, passband, order), cnn.OptimizedZScoreNormalizaton()];



In [4]:

# counts each class
classes_per_recording = np.zeros((nb_of_datasets), dtype='int')
for i in range(0 , nb_of_datasets):
    single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
    gt_data = np.load(single_recording_ground_truth);
    classes_per_recording[i] = np.unique(gt_data[1,:]).size;
    print(np.unique(gt_data[1,:]))
    print(np.min(gt_data))
print(classes_per_recording)

# class counter to concatenate classes together from different recordings
class_counter = np.zeros((nb_of_datasets, 1), dtype='int');
class_counter[0] = 0;
for i in range(1, nb_of_datasets):
    class_counter[i] = class_counter[i-1] + classes_per_recording[i - 1] - 1; 
print(class_counter)



[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
0.0
[0. 1. 2. 3. 4.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.]
0.0
[0. 1. 2.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
0.0
[0. 1. 2. 3.]
0.0
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
0.0
[0. 1. 2. 3. 4. 5. 6. 7. 8.]
0.0
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
0.0
[0. 1. 2. 

In [5]:
# couts sample count for each class
nb_of_classes = int(np.sum(classes_per_recording - 1) + 1);


In [6]:
print(classes_per_recording)
print(class_counter)

[17 20 13  5 16 12 18  3 20 21 21 21 11  4 10  9 10  8 19 15  5  7 18  7
 17 14  6 15  4  6 12 11  5  9 13 19 11 14  3  8 16 10 17 14  9]
[[  0]
 [ 16]
 [ 35]
 [ 47]
 [ 51]
 [ 66]
 [ 77]
 [ 94]
 [ 96]
 [115]
 [135]
 [155]
 [175]
 [185]
 [188]
 [197]
 [205]
 [214]
 [221]
 [239]
 [253]
 [257]
 [263]
 [280]
 [286]
 [302]
 [315]
 [320]
 [334]
 [337]
 [342]
 [353]
 [363]
 [367]
 [375]
 [387]
 [405]
 [415]
 [428]
 [430]
 [437]
 [452]
 [461]
 [477]
 [490]]


In [7]:
16+19+12

47

In [8]:
max_dataset_size = max_dataset_size + nb_of_classes - max_dataset_size % nb_of_classes;
samples_per_class = max_dataset_size // nb_of_classes;
print(samples_per_class)
print(max_dataset_size / nb_of_classes)
print(max_dataset_size)


2005
2005.0
1000495


In [9]:
unique_list = [];
dataset_spikes = np.array([])
labels_spikes = np.array([], dtype = "int")
# adds single unit activity data
for i in range(0, nb_of_datasets):
    added_sample_counter = np.zeros((classes_per_recording[i] - 1));
    data_iter_counter = 0;
    total_nb_of_samples = int((classes_per_recording[i] - 1) * samples_per_class);
    temp_spikes = np.zeros((total_nb_of_samples, 1, waveform_length));
    temp_labels = np.zeros((1, total_nb_of_samples), dtype='int');
    print(dataset_spikes.shape)
    # generates augmentations until there are same sized samples for each class
    while (data_iter_counter < total_nb_of_samples):
        single_recording_path = path.join(path_to_recordings, 'data_' + str(i + 1) + '.npy');
        single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
        generated_spikes = cnn.GenerateDataset(single_recording_path, single_recording_ground_truth, waveform_length, 600, snr_from, snr_to, max_shift, use_horizontal_flip, use_vertical_flip, transform_list);
        # adds to temp variable to add to real dataset afterwards
        for j, (data, target) in enumerate(generated_spikes):
            neuron_index = target.item();
            if(neuron_index > 0 and added_sample_counter[neuron_index - 1] < samples_per_class):
                spike_waveform =  (data.numpy());
                temp_spikes[data_iter_counter, :] = spike_waveform;
                class_index = class_counter[i] + neuron_index;
                # changes class index
                temp_labels[0, data_iter_counter] = class_index;
                added_sample_counter[neuron_index - 1] = added_sample_counter[neuron_index - 1] + 1;
                data_iter_counter = data_iter_counter + 1;
    unique_list.append(np.unique(temp_labels));
    print(added_sample_counter)
    if (dataset_spikes.size == 0):
        dataset_spikes = temp_spikes;
    else:
        dataset_spikes = np.concatenate((dataset_spikes, temp_spikes), axis=0)  
        
    if (labels_spikes.size == 0): 
        labels_spikes = temp_labels;
    else:
        labels_spikes = np.concatenate((labels_spikes, temp_labels), axis=1)            


(0,)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  56.497953003520685
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ede9b5f8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(16, dtype=torch.int32)


/home/vtpc/Documents/Alvils/spike-sorting/custom_resnet/CustomResnet.py:293: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  temp = temp.new_tensor(data);


dataset len:  444296
[2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005.
 2005. 2005. 2005. 2005.]
(32080, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  94.91702933132575
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f160dd4a2b0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(19, dtype=torch.int32)
dataset len:  472786
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ede9b550>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(20, dtype=torch.int32)
dataset len:  554075
[2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005.
 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005.]
(310775, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  38.58478759883971
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ede9b6d8>, <custom_resnet.Custo

[<custom_resnet.CustomResnet.Awgn object at 0x7f15ede9b6a0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(4, dtype=torch.int32)
dataset len:  158693
[2005. 2005. 2005. 2005.]
(515285, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  67.62966769274732
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ede9b438>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
te

dataset len:  304214
[2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005.]
(707765, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  100.32851403304956
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec6f99e8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(10, dtype=torch.int32)
dataset len:  335775
[2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005.]
(727815, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
s

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec6f99e8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(15, dtype=torch.int32)
dataset len:  493728
[2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005.
 2005. 2005. 2005.]
(906260, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  60.15899710070044
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec6f99e8>, <custom_resnet.CustomResnet.FilterSignalUsingButte

In [10]:

dataset_multiunit_spikes = np.array([])
labels_multiunit_spikes = np.array([], dtype = "int")
    
# adds multi unit activity data
data_iter_counter = 0;
total_nb_of_samples = int(labels_spikes.size);
temp_spikes = np.zeros((total_nb_of_samples, 1, waveform_length));
temp_labels = np.zeros((1, total_nb_of_samples), dtype='int');
while (data_iter_counter < total_nb_of_samples):
    for i in range(0, nb_of_datasets):
        single_recording_path = path.join(path_to_recordings, 'data_' + str(i + 1) + '.npy');
        single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
        generated_spikes = cnn.GenerateDataset(single_recording_path, single_recording_ground_truth, waveform_length, 600, snr_from, snr_to, max_shift, use_horizontal_flip, use_vertical_flip, transform_list);
        # adds to temp variable to add to real dataset afterwards
        for j, (data, target) in enumerate(generated_spikes):
            neuron_index = target.item();
            if(neuron_index == 0 and data_iter_counter < total_nb_of_samples):
                spike_waveform =  (data.numpy());
                temp_spikes[data_iter_counter, :] = spike_waveform;
                class_index = neuron_index;
                # changes class index
                temp_labels[0, data_iter_counter] = class_index;
                data_iter_counter = data_iter_counter + 1;
                
print(added_sample_counter)
if (dataset_multiunit_spikes.size == 0):
    dataset_multiunit_spikes = temp_spikes;
else:
    dataset_multiunit_spikes = np.concatenate((dataset_multiunit_spikes, temp_spikes), axis=0)  
        
if (labels_multiunit_spikes.size == 0): 
    labels_multiunit_spikes = temp_labels;
else:
    labels_multiunit_spikes = np.concatenate((labels_multiunit_spikes, temp_labels), axis=1)          

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  38.91610292776751
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec6f9550>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(16, dtype=torch.int32)
dataset len:  444296
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec6f9ef0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(20, dtype=torch.int32)
dataset len:  498945
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  20.42980452549658
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec6f9550>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160d

dataset len:  494727
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  89.78264946745034
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec6f96d8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(6, dtype=torch.int32)
dataset len:  233026
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec6f9550>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(12, dtype=torch.int32)
dataset len:  350686
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  31.7838442193411
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec6f9978>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd

In [11]:
unique_list = [];
dataset_noise = np.array([])
labels_noise = np.array([], dtype = "int")
    
# adds multi unit activity data
data_iter_counter = 0;
total_nb_of_samples = int(labels_spikes.size);
temp_spikes = np.zeros((total_nb_of_samples, 1, waveform_length));
temp_labels = np.zeros((1, total_nb_of_samples), dtype='int');
while (data_iter_counter < total_nb_of_samples):
    for i in range(0, nb_of_datasets):
        print(i)
        single_recording_path = path.join(path_to_recordings, 'data_' + str(i + 1) + '.npy');
        single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
        noise_indices = cnn.GetNoiseIndices(single_recording_path, single_recording_ground_truth, waveform_length, nb_of_elements);
        noise_class = torch.zeros(1, noise_indices.nelement(), dtype= torch.int);
        noise_data = torch.cat((noise_indices, noise_class), 0);
        np.save(path_to_noise_data, noise_data.numpy());

        generated_spikes = cnn.GenerateDataset(single_recording_path, path_to_noise_data, waveform_length, 600, snr_from, snr_to, max_shift, use_horizontal_flip, use_vertical_flip, transform_list);
        # adds to temp variable to add to real dataset afterwards
        for j, (data, target) in enumerate(generated_spikes):
            neuron_index = target.item();
            if(neuron_index == 0 and data_iter_counter < total_nb_of_samples):
                spike_waveform =  (data.numpy());
                temp_spikes[data_iter_counter, :] = spike_waveform;
                class_index = neuron_index;
                # changes class index
                temp_labels[0, data_iter_counter] = class_index;
                data_iter_counter = data_iter_counter + 1;
                
        
        
unique_list.append(np.unique(temp_labels));
if (dataset_noise.size == 0):
    dataset_noise = temp_spikes;
else:
    dataset_noise = np.concatenate((dataset_noise, temp_spikes), axis=0)  
        
if (labels_noise.size == 0): 
    labels_noise = temp_labels;
else:
    labelspike_trains_noise = np.concatenate((labels_noise, temp_labels), axis=1)            

0
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  54.77065799533314
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec727e80>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
1
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec710518>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
12
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  35.39707352132948
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec710630>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160d

23
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  38.74493276899418
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec710710>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
24
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8, 

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec7105c0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
35
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  39.71596169617593
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec710c88>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160d

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  63.50139790388551
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec70d320>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
2
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,

...

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec70d7f0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
37
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  90.19988277689392
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec70de80>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160d

3
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  32.742531032484635
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec70d438>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
4
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,  

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec70dfd0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
15
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  67.98384821253694
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec70db70>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160d

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  47.72580320375818
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec70d710>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
27
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec70dd68>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
38
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  44.67211903807571
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec70d438>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160d

In [12]:
unique_list = [];
dataset_noise_multiunit = np.array([])
labels_noise_multiunit = np.array([], dtype = "int")
    
# adds multi unit activity data
data_iter_counter = 0;
total_nb_of_samples = int(labels_spikes.size);
temp_spikes = np.zeros((total_nb_of_samples, 1, waveform_length));
temp_labels = np.zeros((1, total_nb_of_samples), dtype='int');
while (data_iter_counter < total_nb_of_samples):
    for i in range(0, nb_of_datasets):
        single_recording_path = path.join(path_to_recordings, 'data_' + str(i + 1) + '.npy');
        single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
        noise_indices = cnn.GetNoiseIndices(single_recording_path, single_recording_ground_truth, waveform_length, nb_of_elements);
        noise_class = torch.zeros(1, noise_indices.nelement(), dtype= torch.int);
        noise_data = torch.cat((noise_indices, noise_class), 0);
        np.save(path_to_noise_data, noise_data.numpy());

        generated_spikes = cnn.GenerateDataset(single_recording_path, path_to_noise_data, waveform_length, 600, snr_from, snr_to, max_shift, use_horizontal_flip, use_vertical_flip, transform_list);
        # adds to temp variable to add to real dataset afterwards
        for j, (data, target) in enumerate(generated_spikes):
            neuron_index = target.item();
            if(neuron_index == 0 and data_iter_counter < total_nb_of_samples):
                spike_waveform =  (data.numpy());
                temp_spikes[data_iter_counter, :] = spike_waveform;
                class_index = neuron_index;
                # changes class index
                temp_labels[0, data_iter_counter] = class_index;
                data_iter_counter = data_iter_counter + 1;
                
        
unique_list.append(np.unique(temp_labels));
if (dataset_noise_multiunit.size == 0):
    dataset_noise_multiunit = temp_spikes;
else:
    dataset_noise_multiunit = np.concatenate((dataset_noise_multiunit, temp_spikes), axis=0)  
        
if (labels_noise_multiunit.size == 0): 
    labels_noise_multiunit = temp_labels;
else:
    labels_noise_multiunit = np.concatenate((labels_noise_multiunit, temp_labels), axis=1)            

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  24.676254026631113
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec70df28>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,


[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec7220b8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  37.56257332142904
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec722438>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  78.91153907365843
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec7228d0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
 

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec7226d8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  78.15060220294467
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec7226d8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  90.44143906151528
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec7226a0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
 

...

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec722ef0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  61.14144375659319
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec722e48>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  28.291040570522576
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec722be0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,


[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec723320>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  82.01291822213858
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec723630>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  30.296898572874547
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec723518>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,


[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec723cf8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a278>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  92.7544863297493
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7f15ec7239e8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7f160dd4a240>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7f160dd4a2

In [13]:
np.save(path_to_train_data, dataset)
np.save(path_to_train_labels, labels)


NameError: name 'dataset' is not defined

In [ ]:
mean = (np.mean(np_data_spikes))
std = (np.std(np_data_spikes))
np.save(path_to_mean_std, [mean, std])

print(mean)
print(std)